# Initialize

In [48]:
%load_ext autoreload
%autoreload 2

#–––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

import torch
from torchvision import datasets, transforms
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch import nn


from AE.models import AE_0, ProgressiveAE
from AE.train import train, layer_wise_pretrain_load_dict, train_ProgressiveAE
from AE.datasets import MNISTDigit2Dataset
from AE.plotter_functions import plot_original_vs_decoded, visualize_bottleneck_features

from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import ExponentialLR

#–––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––


# if torch.backends.mps.is_available():
#     device = torch.device("mps")
#     print("Utilizzo Apple Silicon GPU (MPS)")
# elif torch.cuda.is_available():
#     device = torch.device("cuda")
#     print("Utilizzo NVIDIA GPU (CUDA)")
# else:
#     device = torch.device("cpu")
#     print("Utilizzo la CPU")

device = torch.device("cpu")  # Fallback to CPU if no GPU is available

SEED = 42
torch.manual_seed(SEED)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Datasets


In [23]:

batch_size = 64

## MNIST


train_loader_MNIST = torch.utils.data.DataLoader(
    datasets.MNIST(
        '/Users/enricofrausin/Programmazione/PythonProjects/Fisica/data',
        train=True,
        download=True,
        transform=transforms.ToTensor()
        ),
    batch_size=batch_size,
    shuffle=True
    )

val_loader_MNIST = torch.utils.data.DataLoader(
    datasets.MNIST(
        '/Users/enricofrausin/Programmazione/PythonProjects/Fisica/data',
        train=False,
        download=True,
        transform=transforms.ToTensor()
        ),
    batch_size=batch_size,
    shuffle=False
    )



In [ ]:

## ExtendedMNIST


train_loader_EMNIST = torch.utils.data.DataLoader(
    datasets.EMNIST(
        '/Users/enricofrausin/Programmazione/PythonProjects/Fisica/data',
        split='balanced',
        train=True,
        download=True,
        transform=transforms.ToTensor()
        ),
    batch_size=batch_size,
    shuffle=True
    )

val_loader_EMNIST = torch.utils.data.DataLoader(
    datasets.EMNIST(
        '/Users/enricofrausin/Programmazione/PythonProjects/Fisica/data',
        split='balanced',
        train=False,
        download=True,
        transform=transforms.ToTensor()
        ),
    batch_size=batch_size,
    shuffle=False
    )


## 2MNIST


dataset_2MNIST_train = MNISTDigit2Dataset(train=True, download=True, target_size=60000)
print(f"Dataset size: {len(dataset_2MNIST_train)}")
print(f"Image shape: {dataset_2MNIST_train[0][0].shape}")
print(f"Label: {dataset_2MNIST_train[0][1]}")
train_loader_2MNIST = DataLoader(dataset_2MNIST_train, batch_size=batch_size, shuffle=True)

batch_images, batch_labels = next(iter(train_loader_2MNIST))
print(f"Batch images shape: {batch_images.shape}")
print(f"Batch labels shape: {batch_labels.shape}")
print(f"All labels are 2: {torch.all(batch_labels == 2)}")

print("\n––––––––––––––––––––––––––––––––––––––––––––––––––––––\n")

dataset_2MNIST_val = MNISTDigit2Dataset(train=False, download=True, target_size=10000)
print(f"Dataset size: {len(dataset_2MNIST_train)}")
print(f"Image shape: {dataset_2MNIST_train[0][0].shape}")
print(f"Label: {dataset_2MNIST_train[0][1]}")
print(f"All labels are 2: {torch.all(batch_labels == 2)}")
val_loader_2MNIST = DataLoader(dataset_2MNIST_val, batch_size=batch_size, shuffle=True)

print(f"Batch images shape: {batch_images.shape}")
print(f"Batch labels shape: {batch_labels.shape}")
print(f"All labels are 2: {torch.all(batch_labels == 2)}")



#-------------------------------------------------------------------


datasets = ["MNIST", "EMNIST", "2MNIST"]
train_loaders = {
    "MNIST": train_loader_MNIST,
    "EMNIST": train_loader_EMNIST,
    "2MNIST": train_loader_2MNIST
}
val_loaders = {
    "MNIST": val_loader_MNIST,
    "EMNIST": val_loader_EMNIST,
    "2MNIST": val_loader_2MNIST
}



Found 5958 original samples of digit '2'
Generated 60000 augmented samples
Dataset size: 60000
Image shape: torch.Size([1, 28, 28])
Label: 2
Batch images shape: torch.Size([64, 1, 28, 28])
Batch labels shape: torch.Size([64])
All labels are 2: True

––––––––––––––––––––––––––––––––––––––––––––––––––––––

Found 1032 original samples of digit '2'
Generated 10000 augmented samples
Dataset size: 60000
Image shape: torch.Size([1, 28, 28])
Label: 2
All labels are 2: True
Batch images shape: torch.Size([64, 1, 28, 28])
Batch labels shape: torch.Size([64])
All labels are 2: True


In [ ]:

## FashionMNIST


train_loader_FashionMNIST = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        '/Users/enricofrausin/Programmazione/PythonProjects/Fisica/data',
        train=True,
        download=True,
        transform=transforms.ToTensor()
        ),
    batch_size=batch_size,
    shuffle=True
    )

val_loader_FashionMNIST = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        '/Users/enricofrausin/Programmazione/PythonProjects/Fisica/data',
        train=False,
        download=True,
        transform=transforms.ToTensor()
        ),
    batch_size=batch_size,
    shuffle=False
    )


# 6 features


In [ ]:
for dataset in datasets:
    print("\n--------------------------------")
    print(f"TRAINING ON {dataset}")
    print("--------------------------------\n")
    train_loader = train_loaders[dataset]
    val_loader = val_loaders[dataset]

    for num_hidden_layers in range(1, 8):
        print(f"\n--- Training ProgressiveAE with {num_hidden_layers} hidden layers ---\n")

        latent_dim=6
        decrease_rate=0.7
        input_dim = 784
        learning_rate = 1e-3

        my_model = ProgressiveAE(
            input_dim=input_dim,
            latent_dim=latent_dim,
            decrease_rate=decrease_rate,
            device=device,
            num_hidden_layers=num_hidden_layers
        ).to(device)

        log_dir = f'/Users/enricofrausin/Programmazione/PythonProjects/Tesi/Autoencoders/runs/progressiveAE/{dataset}/ld{latent_dim}_dr{decrease_rate}_lr1e3_3epEach_keepTrain_{num_hidden_layers}hl'
        writer = SummaryWriter(log_dir=log_dir)

        train_ProgressiveAE(
            my_model,
            epochs_for_each_neuron=3,
            train_loader=train_loader,
            val_loader=val_loader,
            writer=writer,
            freeze_prev_neurons_train=False
        )

        model_path = f'/Users/enricofrausin/Programmazione/PythonProjects/Tesi/Autoencoders/models/progressiveAE/{dataset}/ld{latent_dim}_dr{decrease_rate}_lr1e3_3epEach_keepTrain_{num_hidden_layers}hl.pth'
        torch.save(my_model.state_dict(), model_path)



# 8 features

In [ ]:
print("\n--------------------------------")
print("TRAINING ON MNIST")
print("--------------------------------\n")
train_loader = train_loader_MNIST
val_loader = val_loader_MNIST
input_dim = 784

learning_rate = 1e-3
weight_decay = 1e-5



for num_hidden_layers in range(1, 8):
    print(f"\n--- Training ProgressiveAE with {num_hidden_layers} hidden layers ---\n")

    dataset = "MNIST"
    latent_dim = 8
    decrease_rate = 0.7
    input_dim = 784
    learning_rate = 1e-3

    my_model = ProgressiveAE(
        input_dim=input_dim,
        latent_dim=latent_dim,
        decrease_rate=decrease_rate,
        device=device,
        num_hidden_layers=num_hidden_layers
    ).to(device)

    log_dir = f'/Users/enricofrausin/Programmazione/PythonProjects/Tesi/Autoencoders/runs/progressiveAE/{dataset}/ld{latent_dim}_dr{decrease_rate}_lr1e3_3epEach_keepTrain_{num_hidden_layers}hl'
    writer = SummaryWriter(log_dir=log_dir)

    train_ProgressiveAE(
        my_model,
        epochs_for_each_neuron=3,
        train_loader=train_loader,
        val_loader=val_loader,
        writer=writer,
        freeze_prev_neurons_train=False
    )

    model_path = f'/Users/enricofrausin/Programmazione/PythonProjects/Tesi/Autoencoders/models/progressiveAE/{dataset}/ld{latent_dim}_dr{decrease_rate}_lr1e3_3epEach_keepTrain_{num_hidden_layers}hl.pth'
    torch.save(my_model.state_dict(), model_path)


--------------------------------
TRAINING ON MNIST
--------------------------------


--- Training ProgressiveAE with 1 hidden layers ---


-----------------------------TRAINING STARTED---------------------------- 

 ------------ Training of neuron1---------------
Epoch: 0/3, Average loss: 0.0012
Epoch: 1/3, Average loss: 0.0012
Epoch: 2/3, Average loss: 0.0011
Training of neuron 1 completed. Final training loss: 0.0010701680670802793, Validation loss: 0.0009825356785207987

 ------------ Training of neuron2---------------
Epoch: 0/3, Average loss: 0.0009
Epoch: 1/3, Average loss: 0.0009
Epoch: 2/3, Average loss: 0.0009
Training of neuron 2 completed. Final training loss: 0.0008820286325489481, Validation loss: 0.0008258287854492664

 ------------ Training of neuron3---------------
Epoch: 0/3, Average loss: 0.0008
Epoch: 1/3, Average loss: 0.0007
Epoch: 2/3, Average loss: 0.0007
Training of neuron 3 completed. Final training loss: 0.0006934997223938504, Validation loss: 0.000684979897

In [49]:
print("\n--------------------------------")
print("TRAINING ON EMNIST")
print("--------------------------------\n")
train_loader = train_loader_EMNIST
val_loader = val_loader_EMNIST



for num_hidden_layers in range(1, 8):
    print(f"\n--- Training ProgressiveAE with {num_hidden_layers} hidden layers ---\n")

    dataset = "EMNIST"
    latent_dim = 8
    decrease_rate = 0.7
    input_dim = 784
    learning_rate = 1e-3

    my_model = ProgressiveAE(
        input_dim=input_dim,
        latent_dim=latent_dim,
        decrease_rate=decrease_rate,
        device=device,
        num_hidden_layers=num_hidden_layers
    ).to(device)

    log_dir = f'/Users/enricofrausin/Programmazione/PythonProjects/Tesi/Autoencoders/runs/progressiveAE/{dataset}/ld{latent_dim}_dr{decrease_rate}_lr1e3_3epEach_keepTrain_{num_hidden_layers}hl'
    writer = SummaryWriter(log_dir=log_dir)

    train_ProgressiveAE(
        my_model,
        epochs_for_each_neuron=3,
        train_loader=train_loader,
        val_loader=val_loader,
        writer=writer,
        freeze_prev_neurons_train=False
    )

    model_path = f'/Users/enricofrausin/Programmazione/PythonProjects/Tesi/Autoencoders/models/progressiveAE/{dataset}/ld{latent_dim}_dr{decrease_rate}_lr1e3_3epEach_keepTrain_{num_hidden_layers}hl.pth'
    torch.save(my_model.state_dict(), model_path)





--------------------------------
TRAINING ON EMNIST
--------------------------------


--- Training ProgressiveAE with 1 hidden layers ---


-----------------------------TRAINING STARTED---------------------------- 

 ------------ Training of neuron1---------------
Epoch: 0/3, Average loss: 0.0014
Epoch: 1/3, Average loss: 0.0013
Epoch: 2/3, Average loss: 0.0012
Training of neuron 1 completed. Final training loss: 0.001225642301802728, Validation loss: 0.00122557910040338

 ------------ Training of neuron2---------------
Epoch: 0/3, Average loss: 0.0012
Epoch: 1/3, Average loss: 0.0012
Epoch: 2/3, Average loss: 0.0011
Training of neuron 2 completed. Final training loss: 0.0010902820587329937, Validation loss: 0.001084955767827465

 ------------ Training of neuron3---------------
Epoch: 0/3, Average loss: 0.0011
Epoch: 1/3, Average loss: 0.0010
Epoch: 2/3, Average loss: 0.0010
Training of neuron 3 completed. Final training loss: 0.0009785231495778083, Validation loss: 0.000974678876314

In [50]:


print("\n--------------------------------")
print("TRAINING ON 2MNIST")
print("--------------------------------\n")
train_loader = train_loader_2MNIST
val_loader = val_loader_2MNIST



for num_hidden_layers in range(1, 8):
    print(f"\n--- Training ProgressiveAE with {num_hidden_layers} hidden layers ---\n")

    dataset = "2MNIST"
    latent_dim = 8
    decrease_rate = 0.7
    input_dim = 784
    learning_rate = 1e-3

    my_model = ProgressiveAE(
        input_dim=input_dim,
        latent_dim=latent_dim,
        decrease_rate=decrease_rate,
        device=device,
        num_hidden_layers=num_hidden_layers
    ).to(device)

    log_dir = f'/Users/enricofrausin/Programmazione/PythonProjects/Tesi/Autoencoders/runs/progressiveAE/{dataset}/ld{latent_dim}_dr{decrease_rate}_lr1e3_3epEach_keepTrain_{num_hidden_layers}hl'
    writer = SummaryWriter(log_dir=log_dir)

    train_ProgressiveAE(
        my_model,
        epochs_for_each_neuron=3,
        train_loader=train_loader,
        val_loader=val_loader,
        writer=writer,
        freeze_prev_neurons_train=False
    )

    model_path = f'/Users/enricofrausin/Programmazione/PythonProjects/Tesi/Autoencoders/models/progressiveAE/{dataset}/ld{latent_dim}_dr{decrease_rate}_lr1e3_3epEach_keepTrain_{num_hidden_layers}hl.pth'
    torch.save(my_model.state_dict(), model_path)


--------------------------------
TRAINING ON 2MNIST
--------------------------------


--- Training ProgressiveAE with 1 hidden layers ---


-----------------------------TRAINING STARTED---------------------------- 

 ------------ Training of neuron1---------------
Epoch: 1/3, Average loss: 0.0015
Epoch: 2/3, Average loss: 0.0014
Epoch: 3/3, Average loss: 0.0014
Training of neuron 1 completed. Final training loss: 0.0013949973892420531, Validation loss: 0.0014048923671245576

 ------------ Training of neuron2---------------
Epoch: 1/3, Average loss: 0.0012
Epoch: 2/3, Average loss: 0.0012
Epoch: 3/3, Average loss: 0.0012
Training of neuron 2 completed. Final training loss: 0.0011607789353777966, Validation loss: 0.0010787366136908532

 ------------ Training of neuron3---------------
Epoch: 1/3, Average loss: 0.0011
Epoch: 2/3, Average loss: 0.0010
Epoch: 3/3, Average loss: 0.0010
Training of neuron 3 completed. Final training loss: 0.001035028750014802, Validation loss: 0.001026984667

# 10 features


In [ ]:
for dataset in datasets:
    print("\n--------------------------------")
    print(f"TRAINING ON {dataset}")
    print("--------------------------------\n")
    train_loader = train_loaders[dataset]
    val_loader = val_loaders[dataset]

    for num_hidden_layers in range(1, 8):
        print(f"\n--- Training ProgressiveAE with {num_hidden_layers} hidden layers ---\n")

        latent_dim=10
        decrease_rate=0.7
        input_dim = 784
        learning_rate = 1e-3

        my_model = ProgressiveAE(
            input_dim=input_dim,
            latent_dim=latent_dim,
            decrease_rate=decrease_rate,
            device=device,
            num_hidden_layers=num_hidden_layers
        ).to(device)

        log_dir = f'/Users/enricofrausin/Programmazione/PythonProjects/Tesi/Autoencoders/runs/progressiveAE/{dataset}/ld{latent_dim}_dr{decrease_rate}_lr1e3_3epEach_keepTrain_{num_hidden_layers}hl'
        writer = SummaryWriter(log_dir=log_dir)

        train_ProgressiveAE(
            my_model,
            epochs_for_each_neuron=3,
            train_loader=train_loader,
            val_loader=val_loader,
            writer=writer,
            freeze_prev_neurons_train=False
        )

        model_path = f'/Users/enricofrausin/Programmazione/PythonProjects/Tesi/Autoencoders/models/progressiveAE/{dataset}/ld{latent_dim}_dr{decrease_rate}_lr1e3_3epEach_keepTrain_{num_hidden_layers}hl.pth'
        torch.save(my_model.state_dict(), model_path)
